In [26]:
import pandas as pd
import mlflow.pytorch
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from graph_reinforcement_learning_using_blockchain_data import config
from graph_reinforcement_learning_using_blockchain_data.modeling import gnn

config.load_dotenv()

True

In [2]:
dataset = torch.load(config.FLASHBOTS_Q2_DATA_DIR / "state_graphs.pt", weights_only=False)

In [3]:
max_feats_len = 0
for vals in dataset.values():
    for graphs in vals:
        if graphs.x.shape[1] > max_feats_len:
            max_feats_len = graphs.x.shape[1]

In [4]:
def pad_features(x, max_length):
    # x is a tensor of shape [num_nodes, current_feature_dim]
    current_dim = x.size(1)
    if current_dim < max_length:
        pad_amount = max_length - current_dim
        # F.pad expects pad widths for the last dimensions: (pad_left, pad_right)
        # Here we pad only on the right of dimension 1.
        x = F.pad(x, (0, pad_amount), "constant", 0)
    return x

In [5]:
for vals in dataset.values():
    for graphs in vals:
        graphs.x = pad_features(graphs.x, max_feats_len)

In [6]:
dataset["0x0000000000055772357E58581A2A2C5C6D9E8F64"][5].x

tensor([[   6.0000,   40.3639,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000],
        [  18.0000,    0.0000,    0.0000,    1.2286,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0

In [14]:
data_list = [d for sublist in dataset.values() for d in sublist]

generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(
    data_list, [0.8, 0.2], generator=generator
)

In [15]:
train_loader = DataLoader(
    train_dataset, batch_size=256, shuffle=True, exclude_keys=["account_mapping"], drop_last=False
)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)
data_loader = DataLoader(
    data_list, batch_size=256, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)

In [16]:
num_node_features = max_feats_len
dim_global_features = 0
hidden_channels = 256  # adjust as needed
num_classes = 2  # binary classification

In [17]:
model_GNNSAGE = gnn.GraphSAGE(
    num_node_features + dim_global_features, hidden_channels, num_classes
)
print(model_GNNSAGE)

GraphSAGE(
  (conv1): SAGEConv(51, 256, aggr=mean)
  (conv2): SAGEConv(256, 256, aggr=mean)
  (conv3): SAGEConv(256, 256, aggr=mean)
  (conv4): SAGEConv(256, 256, aggr=mean)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [18]:
optimizer = torch.optim.Adam(model_GNNSAGE.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device("mps")
model_GNNSAGE.to(device)

GraphSAGE(
  (conv1): SAGEConv(51, 256, aggr=mean)
  (conv2): SAGEConv(256, 256, aggr=mean)
  (conv3): SAGEConv(256, 256, aggr=mean)
  (conv4): SAGEConv(256, 256, aggr=mean)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [19]:
gnn.run_experiment(
    "Graph SAGE State Embeddings",
    20,
    model_GNNSAGE,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
)

2025/03/30 11:45:15 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 starts


  5%|▌         | 1/20 [03:48<1:12:22, 228.58s/it]

Epoch: 001, Train Loss: 0.0441, Test Acc: 0.9945
Epoch 2 starts


 10%|█         | 2/20 [06:21<55:12, 184.03s/it]  

...

100%|██████████| 20/20 [1:18:23<00:00, 235.20s/it]

Epoch: 020, Train Loss: 0.0142, Test Acc: 0.9967



2025/03/30 13:03:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run debonair-sloth-11 at: http://127.0.0.1:8080/#/experiments/748752183556303764/runs/465d6f94d00b4fe2bef4d8885ed7be39
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/748752183556303764


{}

In [23]:
model_uri = "mlflow-artifacts:/748752183556303764/465d6f94d00b4fe2bef4d8885ed7be39/artifacts/model"
model_GNNSAGE = mlflow.pytorch.load_model(model_uri)

In [22]:
_, _, embeddings = gnn.test(model_GNNSAGE, data_loader, criterion, device, return_embeddings=True)

In [24]:
emb = {trx_id: emb.cpu().detach().numpy() for trx_id, emb in embeddings.items()}

In [27]:
df_embeddings = pd.DataFrame({"transactionHash": emb.keys(), "embeddings": emb.values()})

In [28]:
df_embeddings.to_csv(config.FLASHBOTS_Q2_DATA_DIR / "state_embeddings.csv", index=False)

In [29]:
df_embeddings.set_index("transactionHash", inplace=True)